## Príklad z minula

- Kapacita kondenzátora ako funkcia vzdialenosti:

## $C = \frac{\epsilon A}{d}$

- Zmerané hodnoty:

|   d [cm]   |    C [$\mu$F]    |
|-------|---------|
|  2.00 |  275.4 $\pm$ 10 |
|  3.00 |  182.1 $\pm$ 10  |
|  5.00 |  111.0 $\pm$ 10  |
|  7.00 |   83.2 $\pm$ 10  |
|  9.00 |   68.0 $\pm$ 10  |
| 11.00 |   57.9 $\pm$ 10  |
| 13.00 |   51.2 $\pm$ 10  |
| 15.00 |   45.9 $\pm$ 10  |
| 17.00 |   41.9 $\pm$ 10  |
| 19.00 |   39.0 $\pm$ 10  |
| 20.00 |   37.9 $\pm$ 10  |


- Chceme zistiť konštantu úmery $\epsilon A$ 
- Treba nafitovať $C(\frac{1}{d})$

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy
plt.style.use("default")

In [7]:
C = np.array([275.4, 182.1, 111.0, 83.2, 68.0, 57.9, 51.2, 45.9, 41.9, 39.0, 37.9])
d = np.array([2.00, 3.00, 5.00, 7.00, 9.00, 11.0, 13.0, 15.0, 17.0, 19.0, 20.0])
d_inv = 1 / d
C_err = np.array([10]*len(C))

In [8]:
def lin_func(x, m, c):
   return m * x + c

In [14]:
def plot(x_graph, y_graph, y_err, x_fit, y_fit, y_fit_lower, y_fit_upper, xlabel, filename, ylabel=r"C [$\mu$F]"):

    # draw x_graph, y_graph with error bars
    plt.errorbar(x_graph, y_graph, y_err, fmt='o', label='Data', color='black')

    # draw the fit function and its uncertainty band
    plt.fill_between(x_fit, y_fit_lower, y_fit_upper, color='red', alpha=0.3)

    # create a legend entry for the fit function and its uncertainty band
    line_with_band = mpl.lines.Line2D([], [], color='red', label='Fit', linestyle='-', linewidth=2)
    band = mpl.patches.Patch(color='red', alpha=0.3, label='Fit uncertainty')

    # get the current legend handles and labels
    handles, labels = plt.gca().get_legend_handles_labels()
    plt.legend(handles=handles + [(line_with_band, band)], labels=labels + ['Fit'])

    # finally, plot
    plt.plot(x_fit, y_fit, 'r-')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.savefig(filename, dpi=300)
    plt.show() # display the plot in jupyter notebook
    plt.close()

    return

In [ ]:
popt, pcov = scipy.optimize.curve_fit(lin_func, d_inv, C, sigma=C_err, absolute_sigma=True)
x_fit = np.linspace(min(d_inv), max(d_inv), 100)
y_fit = lin_func(x_fit, popt[0], popt[1])
# error propagation; take into account the uncertainty of the two fit parameters and their correlation
dyda = x_fit
dydb = 1
y_unc = np.sqrt(dyda**2 * pcov[0,0] + dydb**2 * pcov[1,1] + 2*dyda*dydb*pcov[0,1])
y_fit_upper = y_fit + y_unc
y_fit_lower = y_fit - y_unc
plot(d_inv, C, C_err, x_fit, y_fit, y_fit_lower, y_fit_upper, "1/d [1/cm]", "linear.png")

## Propagácia chýb
- použijeme magický vzorec na propagáciu chýb
- závislosť $y = y(x_1, x_2, x_3,...)$
- $\vec{\mu} = (\mu_1, \mu_2, ...)$ sú namerané hodnoty veličiny $\vec{x} = (x_1, x_2,...)$
- $\vec{\sigma} = (\sigma_{x_1}, \sigma_{x_2}, ...)$ sú chyby merania veličiny $\vec{x}$
### $\sigma_y = \sqrt{\left(\frac{\partial y}{\partial x_1}\Big|_{\vec x=\vec\mu}\right)^2\sigma_{x_1}^2 + \left(\frac{\partial y}{\partial x_2}\Big|_{\vec x=\vec\mu}\right)^2\sigma_{x_2}^2 + \left(\frac{\partial y}{\partial x_3}\Big|_{\vec x=\vec\mu}\right)^2\sigma_{x_3}^2 + \cdots}$

### Príklad
- $s(a, t) = \frac{1}{2} a t^2$
- $ a = g = 10 \pm 2\ \mathrm{m/s^2}$
- $t = 3 \pm 0.05\ \mathrm{s}$
- $\vec{x} = (a, t)$, $\vec{\mu} = (10\ \mathrm{m/s^2},\ 3\ \mathrm{s})$, $\vec{\sigma} = (2\ \mathrm{m/s^2},\ 0.05\ \mathrm{s})$
- $s = \frac{1}{2} a t^2 = 45\ \mathrm{m} \pm \sigma_s$

$\sigma_s = \sqrt{\left(\frac{\partial s}{\partial a}\Big|_{a=10\ \mathrm{m/s^2}, t=3\ \mathrm{s}}\right)^2\cdot \sigma_a^2 + \left(\frac{\partial s}{\partial t}\Big|_{a=10\ \mathrm{m/s^2}, t=3\ \mathrm{s}}\right)^2\cdot \sigma_t^2} = \sqrt{(\frac{1}{2} t^2\cdot \sigma_a)^2 + (at\cdot \sigma_t)^2} = \sqrt{(\frac{1}{2}3^2\cdot 2)^2 + (10\cdot 3\cdot 0.05)^2} = 9\ \mathrm{m}$

### Všeobecný vzorec s koleráciami
- v prípade, že $x_1, x_2, x_3,...$ nie sú nezávislé, ale sú korelované
### $\sigma_y = \sqrt{\sum\limits_{i,j=1}^n \frac{\partial y}{\partial x_i} \frac{\partial y}{\partial x_j} \Big|_{\vec x=\vec\mu} V_{ij}}$
- $V_{ij}$ je kovariančná matica
- $V_{ij}$ je výstupom z fitu:
  ```python
    popt, pcov = scipy.optimize.curve_fit(f, x, y, sigma=sigma)
    V = pcov
```


# Automatizácia počítania chýb

- základom je knižnica `uncertainties` (alebo `qexpy`)
- dokumentácia: knižnice `uncertainties`: [https://uncertainties.readthedocs.io/en/latest/](https://uncertainties.readthedocs.io/en/latest/)
- treba nainštalovať: 
```bash
    pip install uncertainties
```
- každá premenná (float) je definovaná spoločne s chybou --> nový typ premennej `ufloat`
- výpočty s premennými sú automaticky s chybami

In [24]:
from uncertainties import ufloat

g = ufloat(10, 2)

print(f'g = {g} m/s^2')
print(f'Nominal Value: {g.nominal_value}')
print(f'Standard Deviation: {g.std_dev}')

t = ufloat(3., 0.05)
s = 0.5 * g * t**2
print(f's = {s} m')
print(f's = {s:.3g} m')

g = 10.0+/-2.0 m/s^2
Nominal Value: 10.0
Standard Deviation: 2.0
s = 45+/-9 m
s = 45.0+/-9.1 m


## Príklad 1 
Spočítaje hĺbku studne $h$, ak kameň počujete čľupnúť do vody po čase $t = 2 \pm 0.5\ \mathrm{s}$ od vypustenia.

In [ ]:
# Riešenie píkladu 1

## Pole čisel s chybami
- vytváranie poľa čísel ako `numpy.array` s chybami
- treba použiť `uncertainties.unumpy.uarray`: 
- operácie sú tak ako s `numpy.array`, ale súčsasne sa propagujú chyby
- do `uarray` sa dáva zoznam hodnôt a spoločná chyba všetkých hodnôt, alebo zoznam hodnôt a zoznam chýb

In [29]:
from uncertainties import ufloat, unumpy

# pole hodnôt s rovnakou chybou
y_nominal = [41.2, 39.6, 36.3, 33, 26.3, 16.6]
y_err = 0.1
y = unumpy.uarray(y_nominal, y_err)
print(y)

# pole hodnôt s rôznymi chybami
y_nominal = [41.2, 39.6, 36.3, 33, 26.3, 16.6]
y_err = [0.5, 1.0, 1.1, 1.5, 2.0, 2.5]
y = unumpy.uarray(y_nominal, y_err)
print(y)

# automatická propagácia chýb
y /= 100 # y = y / 100
print(y) 

# vyňatie nominálnych hodnôt a chýb
nominal_values = unumpy.nominal_values(y)
std_devs = unumpy.std_devs(y)
print(f'Nominal Values: {nominal_values}')
print(f'Standard Deviations: {std_devs}')


[41.2+/-0.1 39.6+/-0.1 36.3+/-0.1 33.0+/-0.1 26.3+/-0.1 16.6+/-0.1]
[41.2+/-0.5 39.6+/-1.0 36.3+/-1.1 33.0+/-1.5 26.3+/-2.0 16.6+/-2.5]
[0.41200000000000003+/-0.005 0.396+/-0.01 0.363+/-0.011000000000000001
 0.33+/-0.015 0.263+/-0.02 0.166+/-0.025]
Nominal Values: [0.412 0.396 0.363 0.33  0.263 0.166]
Standard Deviations: [0.005 0.01  0.011 0.015 0.02  0.025]


## Špeciálne funkcie
- na propagáciu chýb sú špeciálne funkcie v knižnici `uncertainties`
- tieto funkcie súčasne počítajú hodnotu a chybu
- prístup je podobný ako v `numpy` ale s chybami, dokážu pracovať s poliami

In [30]:
x = unumpy.uarray([1, 2, 3, 4, 5], 0.1)

unumpy.log(x)
unumpy.exp(x)
unumpy.sqrt(x)
unumpy.sin(x)
unumpy.cos(x)
#...

array([0.5403023058681398+/-0.08414709848078966,
       -0.4161468365471424+/-0.09092974268256818,
       -0.9899924966004454+/-0.014112000805986721,
       -0.6536436208636119+/-0.07568024953079283,
       0.2836621854632263+/-0.09589242746631385], dtype=object)

## Príklad 2

- Nafitujte tuhosť pružiny $k$ z merania závislosti závažia $m$ na predĺžení pružiny $y$.
- Predlženie pružiny $y_0$ je dané vzťahom 
   ### $y_0 = \frac{g}{k} \cdot m$

- Nech $y_{m0} = 46.4 \pm 0.5\ \mathrm{cm}$ je dĺžka pružiny bez závažia, tj. predlženie dané len váhou pružiny.
- Oznáčme $y(m)$, ako predlženie pružiny s návesom hmotnosti $m$.
- Potom $y(m) = y_0 + y_{m0} = y_{m0} + \frac{g}{k} \cdot m$.
- Zmerané hodnoty aj s chybami:

 ```python
   y_m0 = 46.4 # cm
   y_m0_err = 0.5
   y = [41.2, 39.6, 36.3, 33, 26.3, 16.6] # cm
   y_err = 0.1
   m = [150, 200, 300, 400, 600, 900] # g
   m_err = 0.
```

- Rady:
  - vytvorte pole čísel s chybami
  - skonvertujte jednotky z cm na m a g na kg 
  - nafitujte závislosť $y(m) - y_{m0}$ 
  - fit urobte pomocou `scipy.optimize.curve_fit`, ktorá neprijíma `uarray`, ale treba zvlasť dať nominálne hodnoty $x$,$y$ a chyby $y$
  - zistite konštantu úmery $\frac{g}{k}$

- Zo vzťahu $\omega = \sqrt{\frac{k}{m}}$ určite $\omega$ a jeho chybu pre všetky hodnoty $m$. Odmocninu z čísla vypočítate pomocou `unumpy.sqrt(x)` alebo `x**0.5`
- Za $g$ dosaďte $9.81\ \mathrm{m/s^2}$


In [ ]:
# Riešenie príkladu 2